In [7]:
search_for = 97
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004378557205200195
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 89451461
type: [1, 1, 1, 1, 97] 97
cases of this type: 88529281
10000 14.92171174325103
20000 15.120235046981394
30000 22.08974204611958
40000 56.09518803054445
50000 54.934247385395956
60000 50.33298267304736
70000 51.33342912768281
80000 50.635816397657834
90000 44.0911659784358
100000 44.76415590328522
110000 50.37173611585772
120000 51.561181172853416
130000 52.902116179647706
140000 49.993592727075416
150000 47.67743104656533
160000 45.74954592436275
170000 48.55275944784965
180000 50.103410420291425
190000 49.50208365548774
200000 47.85999906544131
210000 49.444771825523134
220000 50.391144771495824
230000 57.55907346950575
240000 53.2420524432918
250000 46.79339177916063
260000 51.6426890447581
270000 47.501865009936715
280000 50.07105801312137
290000 49.649214375469086
300000 45.57797349153638
310000 48.532095963627434
32

3090000 44.569358334462095
3100000 45.006655512275195
3110000 42.915164941371124
3120000 45.998977500597114
3130000 49.29171404111883
3140000 46.90877680423633
3150000 40.50933554820464
3160000 48.150451361312314
3170000 56.44766068377923
3180000 8.287593456279827
3190000 7.4152154936240855
3200000 4.509369373471379
3210000 6.180954275227507
3220000 16.560184425921474
3230000 52.898120776986374
3240000 44.11317027555844
3250000 36.538502764664585
3260000 44.72544126971847
3270000 37.0490559714825
3280000 44.70886711653307
3290000 45.46384914760883
3300000 44.32089472502174
3310000 42.19749154600141
3320000 45.782147058909835
3330000 40.869343317611346
3340000 38.034191498803764
3350000 38.4739276503017
3360000 39.94452671447875
3370000 39.06467482835834
3380000 40.75576650953131
3390000 42.09788535873118
3400000 43.77027836937092
3410000 42.86274681984534
3420000 45.509738439534885
3430000 43.662939141244394
3440000 47.59971544235886
3450000 41.78667272202916
3460000 43.01187180854026


6210000 36.54879958015268
6220000 39.55547450153089
6230000 41.40776254348445
6240000 37.699247029013335
6250000 41.885142452899515
6260000 41.20393353148228
6270000 43.01446247957135
6280000 40.82782308542952
6290000 42.612579587086195
6300000 48.26902826991299
6310000 26.763606054545665
6320000 16.11445183327657
6330000 17.23258176749085
6340000 16.890324738318284
6350000 14.96838798276919
6360000 47.752295416934935
6370000 43.571902542077105
6380000 39.25988466578495
6390000 42.91970160435505
6400000 44.92272324922168
6410000 43.03868464313865
6420000 41.71555927531737
6430000 46.22081280204863
6440000 36.651813084170065
6450000 11.24228779520003
6460000 13.974930677669551
6470000 13.752663128136774
6480000 14.044579109571734
6490000 39.5120215267192
6500000 48.28540384244209
6510000 42.390984738889635
6520000 41.589330525408144
6530000 43.12581184742268
6540000 40.11910906738965
6550000 42.925601267072686
6560000 35.3610618700097
6570000 40.07220143017054
6580000 40.70888064181913


9330000 39.727563697476086
9340000 39.63572016517686
9350000 39.589699326638566
9360000 37.472531651904504
9370000 39.7762168202145
9380000 38.24534586168826
9390000 42.40275583487261
9400000 48.961583716239986
9410000 40.266832308053395
9420000 44.38455997516924
9430000 38.91413426481307
9440000 38.43851791207089
9450000 42.29585429429037
9460000 36.88884182510237
9470000 35.759033486618996
9480000 35.53898354129727
9490000 37.26456636674797
9500000 36.61361927889762
9510000 35.89423120196715
9520000 37.01521326373353
9530000 36.097493588974416
9540000 31.98832016878822
9550000 40.029365166112136
9560000 47.55583638434529
9570000 5.879554394854804
9580000 6.7412495516823325
9590000 7.160846840894302
9600000 6.65198163695832
9610000 22.011186695665106
9620000 48.604399181707876
9630000 37.311817938076366
9640000 32.5154940984298
9650000 31.41773084106731
9660000 38.358572627014986
9670000 39.401302929262705
9680000 39.38294954176311
9690000 35.70715259709682
9700000 36.004641369558875


12360000 40.52859999154801
12370000 31.401127910817845
12380000 33.44047381283915
12390000 37.79331132278791
12400000 34.0642392685112
12410000 31.9607830001877
12420000 34.93505362180609
12430000 32.872169326620536
12440000 36.82775982370817
12450000 37.613635807199586
12460000 36.325063562361265
12470000 37.140026677532596
12480000 40.47012528690934
12490000 37.39505641124188
12500000 38.94029489070008
12510000 37.48615561717853
12520000 35.37245641110102
12530000 37.73310043329048
12540000 35.64110616396278
12550000 35.897189025627036
12560000 37.73863026180208
12570000 42.402578588663665
12580000 44.655717110658976
12590000 37.91661293277557
12600000 39.28002365875056
12610000 37.8106809810777
12620000 40.93297881422954
12630000 35.250882711277626
12640000 15.795656727828918
12650000 15.184757448028723
12660000 15.852988778010406
12670000 13.933010263571786
12680000 33.76252440594634
12690000 44.96674123713849
12700000 40.51489889179062
12710000 35.45806074690211
12720000 40.678220

...

75260000 4.100166251853526
75270000 3.9597559684645995
75280000 1.2448474244981078
75290000 0.6419733710870942
75300000 0.6411793694632053
75310000 0.6414859232387476
75320000 1.2353072758678794
75330000 4.281965712621669
75340000 3.504374955369009
75350000 3.2395101923381686
75360000 3.410102723251488
75370000 3.5727400603551933
75380000 3.5350399034343165
75390000 3.3985053813727033
75400000 3.310213598176711
75410000 3.3100895563580592
75420000 3.353902072664314
75430000 3.499978971477694
75440000 3.5524961562135817
75450000 3.867715708333565
75460000 4.069556771607094
75470000 4.042506472394149
75480000 3.712680054495725
75490000 3.57110273671178
75500000 3.650649417988916
75510000 3.5635270212832957
75520000 4.015497877001557
75530000 3.6643596520247463
75540000 3.862897817358427
75550000 4.897012249995648
75560000 3.7871082554617126
75570000 3.7126406175134914
75580000 5.071538014333824
75590000 4.677956154829582
75600000 1.7467494596597684
75610000 1.1755546782760753
75620000 1.

78250000 2.9932027067031597
78260000 3.0518458773259125
78270000 3.230751486594617
78280000 3.2343290431589486
78290000 3.8827466654863363
78300000 2.9452815902877645
78310000 3.321010546083338
78320000 3.276766135064721
78330000 3.0558794315709275
78340000 2.7921535955523256
78350000 3.2203830236282744
78360000 2.99699210414856
78370000 0.9266961889569959
78380000 0.9309643385416267
78390000 0.931051564968288
78400000 0.9236374578781988
78410000 2.0690346930947903
78420000 4.020063429388748
78430000 3.007041585924248
78440000 3.402785262869736
78450000 3.093322800002661
78460000 3.139463500036054
78470000 2.983477841900726
78480000 3.16632253015091
78490000 3.724967025599473
78500000 3.6234882062042946
78510000 3.1869830153924625
78520000 3.0222488830697336
78530000 2.985173853808899
78540000 2.9644797498775457
78550000 3.1952948866319653
78560000 3.218981995678902
78570000 2.1152165643056366
78580000 1.1362601527462073
78590000 1.1333809098417758
78600000 1.0977400805506174
78610000 

81220000 2.469800703567869
81230000 2.3388689031769836
81240000 2.391327609594855
81250000 2.243295926315599
81260000 2.0308118473781946
81270000 2.307931212386462
81280000 1.7755980623045564
81290000 0.8528712881942987
81300000 0.8580976185636653
81310000 0.8373534161125752
81320000 0.8412657736934225
81330000 2.4387506703202924
81340000 2.82862514128785
81350000 2.090141335876465
81360000 2.2830954027475054
81370000 2.473317244294769
81380000 2.0880919313469533
81390000 2.114137310464932
81400000 2.300716965137786
81410000 2.217145029636959
81420000 2.319272653030885
81430000 2.1212098626572553
81440000 1.9516493952497644
81450000 2.1404061575032407
81460000 2.1177146103307534
81470000 1.9647061631506084
81480000 2.02991483218105
81490000 2.401566598250674
81500000 2.1521902213476696
81510000 2.154128735176193
81520000 2.077787793904417
81530000 2.0779741901851296
81540000 2.1822597113562026
81550000 2.2033869671835764
81560000 2.000751471220851
81570000 1.991017352737546
81580000 2.

84180000 1.4998671037499773
84190000 1.4230044782352316
84200000 1.4139279580788213
84210000 1.485866140116122
84220000 1.4449531733661627
84230000 1.8802570162717305
84240000 1.4341878343324397
84250000 1.349847061890761
84260000 1.334476108741045
84270000 1.3536312886915538
84280000 1.3485085744334262
84290000 1.3603019700861374
84300000 1.2848383070958522
84310000 1.4128324651252298
84320000 1.5028953581246733
84330000 1.3823022885797023
84340000 1.6491095188218023
84350000 1.882865573796809
84360000 1.3853398615833123
84370000 1.3596473891637262
84380000 1.3985932852525313
84390000 1.2933424571418761
84400000 1.6302601068928575
84410000 0.23110617369238534
84420000 0.385781129881627
84430000 0.24087198618996145
84440000 0.22400928655308486
84450000 0.8687326836406192
84460000 1.4118278709268042
84470000 1.408842743843158
84480000 1.3290483462946878
84490000 1.3391627795240417
84500000 2.3042991464995546
84510000 1.3910951754545833
84520000 1.3232317792903185
84530000 1.269907384265

87120000 1.0372668817733592
87130000 0.7848500253520607
87140000 0.5732248749359845
87150000 0.10399268061560392
87160000 0.10340298424628044
87170000 0.10296892535897095
87180000 0.10271828902773725
87190000 0.5453039215513468
87200000 0.7463036453888814
87210000 0.641643495776693
87220000 0.7547701983196603
87230000 0.6068026802963019
87240000 0.5893490023004148
87250000 0.5826897135536074
87260000 0.6006768454920848
87270000 0.6022621283796562
87280000 0.613735582364685
87290000 0.6049888853107691
87300000 0.5741940136298869
87310000 0.5594529687761399
87320000 0.7015461533666253
87330000 0.5604206712095605
87340000 0.6201648873045312
87350000 0.5947689218282303
87360000 0.5537680088263685
87370000 0.5806898843498495
87380000 0.5954182570980986
87390000 0.617763193969422
87400000 0.6588296991114285
87410000 0.5864507642610669
87420000 0.6011903567467001
87430000 0.5892323035674957
87440000 0.6949955547013481
87450000 0.6441211559022864
87460000 0.5481676914569206
87470000 0.54758046